In [15]:
import os
import json
import cv2
import albumentations as A

# val set resize & aug

In [16]:
def augment_images_with_bbox(image_dir, annotation_file, output_dir):
    # Load JSON annotation file
    with open(annotation_file, 'r') as f:
        annotations = json.load(f)

    # Define augmentation pipeline
    transform = A.Compose([
        A.OneOf([
            A.Blur(blur_limit=3, p=1.0),
            A.MotionBlur(blur_limit=11, p=1.0),
            A.MedianBlur(blur_limit=3, p=1.0),
        ], p=0.8),
        A.OneOf([
            A.ISONoise(p=1.0),
            A.RandomGamma(p=1.0),
            A.HueSaturationValue(p=1.0),
            A.ChannelShuffle(p=1.0),
            A.RGBShift(p=1.0),
            A.CLAHE(clip_limit=(1, 10), p=1.0),
            A.RandomBrightnessContrast(p=1.0), 
        ], p=1),
    ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

    # Iterate over images and annotations
    for image_data in annotations['images']:
        image_id = image_data['id']
        image_path = os.path.join(image_dir, image_data['file_name'])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        bboxes = []
        category_ids = []

        # Find annotations for the current image
        for annotation in annotations['annotations']:
            if annotation['image_id'] == image_id:
                bboxes.append(annotation['bbox'])
                category_ids.append(annotation['category_id'])

        # Apply augmentation
        augmented = transform(image=image, bboxes=bboxes, category_ids=category_ids)
        augmented_image = augmented['image']

        # Save augmented image
        output_image_path = os.path.join(output_dir, f"augmented_{os.path.basename(image_data['file_name'])}")
        cv2.imwrite(output_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    print("Augmentation completed.")

In [17]:
image_dir = '/opt/ml/DACON-objectdetection/data'
annotation_file = '/opt/ml/DACON-objectdetection/data/K-fold_val1.json'
output_dir = '/opt/ml/DACON-objectdetection/data/val'

In [18]:
augment_images_with_bbox(image_dir, annotation_file, output_dir)

Augmentation completed.


# json file의 image path 변경

In [12]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

for image_data in annotations['images']:
    image_data['file_name'] = image_data['file_name'].replace("train/", "val/augmented_")
        
with open('/opt/ml/DACON-objectdetection/data/augmented_val.json', 'w') as f:
    json.dump(annotations, f)

# CoarseDropout aug 적용

In [19]:
from glob import glob

img_path = '/opt/ml/DACON-objectdetection/data/val/*.png'
img_pathes = glob(img_path)

In [20]:
# Define augmentation pipeline
transform = A.Compose([
    A.CoarseDropout(always_apply=False, p=0.6, min_holes=1, max_holes=5, min_height=8, max_height=150, min_width=8, max_width=20, fill_value=(200,200,215))
])

# Iterate over images
for img_path in img_pathes:
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
    # Apply augmentation
    augmented = transform(image=image)
    augmented_image = augmented['image']
    
    # Save augmented image
    cv2.imwrite(img_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
